In [4]:
import json
import pandas as pd

In [2]:
embeddings = json.load(open("../Data/embeddings.json"))

In [ ]:
embedding_pd = pd.read_parquet("s3://s3-nl-prd-semrb-emr/embeddings/doc_embeddings/word2vec/word2vec_doc_embedding.parquet")
embedding_pd['cv_id'] = embedding_pd['cv_id'].astype('int')
embedding_pd.rename(columns={"doc_embedding": "tensor"}, inplace=True)

In [15]:
emb_df = pd.read_csv("../Data/cv_w2v_embeddings.csv").drop("Unnamed: 0", axis=1)

In [16]:
emb_df.head()

,candidate_id,date_start,embedding
0,84556.0,2020-07-02,[ 0.07265972 -0.85820568 -0.26867145 0.543308...
1,84612.0,2020-05-30,[ 3.47467571e-01 -1.04299009e+00 -3.20929319e-...
2,84612.0,2020-06-03,[ 3.40685010e-01 -1.01796389e+00 -3.20944011e-...
3,84731.0,2020-06-03,[ 0.12455235 -0.17254153 -0.4887622 -0.004285...
4,85437.0,2020-06-06,[ 2.04800099e-01 -4.90688115e-01 -1.86530575e-...


In [ ]:
def add_texts(candidate_work_experience, cv_embeddings):
    
    print("merging...")
    # Add CVs to candidates (includes duplicates)
    full_merge = pd.merge(candidate_work_experience,
                          cv_embeddings,
                          left_on="candidate_id",
                          right_on="candidate_id",
                          how="left")
        
    print("Finding maxima")
    # Find the maximum date of uploaded cvs per job (i.e. most recent CV during each job)
    most_recent_cvs = full_merge[(full_merge["date_start_job"]
                                  >= full_merge["date_start"])]
    
    idx = most_recent_cvs.groupby("unique_id")["date_start"].transform(max) == most_recent_cvs["date_start"]   
    
    most_recent_cvs = most_recent_cvs[idx]
    
    print("Remove late CVs")
    # Some candidates only added CVs after their last job started, so account for that too
    late_cvs = full_merge[(full_merge["date_start_job"] <= 
                           full_merge["date_start"])]
    
    idx = late_cvs.groupby("unique_id")["date_start"].transform(max) == late_cvs["date_start"]
    late_cvs = late_cvs[idx]
    
    display(late_cvs)
    
    print("Clean and combine")
    # Filter out everything we already found earlier
    late_cvs = late_cvs.loc[set(late_cvs.index) - set(most_recent_cvs.index)]
    
    # Now we have all the CVs in one place
    cv_idxs = pd.concat([most_recent_cvs, late_cvs])
    
    # Combine the two frames to include candidates without any CVs
    combined = pd.concat([full_merge[full_merge["text"].isna()], 
                          full_merge.loc[cv_idxs.values]])
    
    return combined

candidate_work_experience = add_texts(candidate_work_experience, cv_df)